### Purpose - Take Weights and Chars Reports from Factset and convert into a Seismic Friendly flat file format.

In [3]:
# Import dependencies
import numpy as np
import pandas as pd
import xlsxwriter

### Step 1:  Wrangle and transform `Sector` data from FactSet

In [4]:
# Import sector weights tab and create df
df = pd.read_excel('input_template.xlsx', sheet_name = 'Sector_Raw', skiprows=7, header= None, names=['Sector', 'SimPortWeight', 'IndexWeight'])

# Drop blank rows
df = df.dropna(axis=0)

# Delete Total Row
df = df[df['Sector'] != 'Total']

# Divide Weights by 100
df[['SimPortWeight', 'IndexWeight']] = df[['SimPortWeight', 'IndexWeight']].div(100)

# Import same file to wo skipping rows to get some add'l necessary meta_data
df_metadata = pd.read_excel('input_template.xlsx', sheet_name = 'Sector_Raw')

# Get report date
rpt_date = df_metadata.iloc[4,0]

# Get report currency
rpt_curr = df_metadata.iloc[0,0]

# Get strategy code
rpt_code = df_metadata.columns[0].split()[0].split('_')[0]

# Insert add'l cols needed for Seismic
df.insert(0, 'AsOfDate', rpt_date)
df.insert(1, 'StrategyCode', rpt_code)

# Preview df and metadata
print('Date: ', rpt_date, " ", "Date Type: ", type(rpt_date), ' Strategy Code: ', rpt_code, \
    ' Currency: ', rpt_curr)
df.head(13)

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

NameError: name 'strategy_code' is not defined

### Step 2:  Wrangle and transform `Market Cap` data from FactSet

In [ ]:
# Import mcap weights tab and create df
mcap = pd.read_excel('input_template.xlsx', sheet_name = 'Mcap_Raw', header = None, skiprows = 7, names= ('Label', 'SimWeight', 'IndexWeight'))

# Insert add'l cols needed for Seismic
mcap.insert(0, 'AsOfDate', rpt_date)
mcap.insert(1, 'StrategyCode', rpt_code)
mcap.insert(2, 'Currency', rpt_curr)

# Drop blank rows
mcap = mcap.dropna(axis=0)

# Delete Total Row
mcap = mcap[mcap['Label'] != 'Total']

# Insert a 'Range' col that maps ints for each bucket
#Length of mcap = 6, np.arrange gives an array starting at 0 and up to but not including length value.
range  = np.arange(len(mcap))
range += 1
mcap.insert(3, 'Range', range)

# Divide weights by 100
mcap[['SimWeight', 'IndexWeight']] = mcap[['SimWeight', 'IndexWeight']].div(100)
mcap

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Currency,Range,Label,SimWeight,IndexWeight
0,9/30/2019,MSAUM,AUD,1,Over 100B,0.271757,0.343094
1,9/30/2019,MSAUM,AUD,2,25B - 100B,0.369919,0.309234
2,9/30/2019,MSAUM,AUD,3,15B - 25B,0.102557,0.114275
3,9/30/2019,MSAUM,AUD,4,2B - 15B,0.255768,0.233397
4,9/30/2019,MSAUM,AUD,5,Under 2B,0.000000,0.000048


In [ ]:
# Import library and create currency symbols to be used in mkt cap output
import unicodedata

EUR_Code = '{}'.format(unicodedata.lookup("EURO SIGN"))
GBP_Code = '{}'.format(unicodedata.lookup('POUND SIGN'))
NZD_Code = 'NZD'
DKK_Code = 'DKK'
USD_Code = '$'
AUD_Code = 'A$'

In [ ]:
# Create and run function to identify appropriate curr symbol for output

def currency_symbol(currency):
    if currency == 'USD':
        return USD_Code
    elif currency == 'GBP':
        return GBP_Code
    elif currency == 'DKK':
        return DKK_Code
    elif currency == 'NZD':
        return NZD_Code
    elif currency == 'AUD':
        return AUD_Code
    elif currency == 'EUR':
        return EUR_Code    

currency_sign = currency_symbol(rpt_curr)
currency_sign

'A$'

In [ ]:
# Input currency sign into mkt cap bucket categories

Mega_Cap = '> {0}100B'.format(currency_sign)
Big_Large_Cap = '{0}25B - {0}100B'.format(currency_sign)
Large_Cap = '{0}15B - {0}25B'.format(currency_sign)
Mid_Cap = '{0}2B - {0}15B'.format(currency_sign)
Small_Cap = '< {0}2B'.format(currency_sign)

In [ ]:
# Replace each original mkt cap bucket to equivalent name w/ the curr sign

mcap['Label'] = mcap['Label'].replace('Over 100B', Mega_Cap)
mcap['Label'] = mcap['Label'].replace('25B - 100B', Big_Large_Cap)
mcap['Label'] = mcap['Label'].replace('15B - 25B', Large_Cap)
mcap['Label'] = mcap['Label'].replace('2B - 15B', Mid_Cap)
mcap['Label'] = mcap['Label'].replace('Under 2B', Small_Cap)

In [ ]:
# Preview final mkt cap df
mcap

,AsOfDate,StrategyCode,Currency,Range,Label,SimWeight,IndexWeight
0,9/30/2019,MSAUM,AUD,1,> A$100B,0.271757,0.343094
1,9/30/2019,MSAUM,AUD,2,A$25B - A$100B,0.369919,0.309234
2,9/30/2019,MSAUM,AUD,3,A$15B - A$25B,0.102557,0.114275
3,9/30/2019,MSAUM,AUD,4,A$2B - A$15B,0.255768,0.233397
4,9/30/2019,MSAUM,AUD,5,< A$2B,0.000000,0.000048


### Step 3:  Wrangle and transform `Characteristics` data from FactSet 

In [ ]:
# Import chars tab and create df
chars = pd.read_excel('input_template.xlsx', sheet_name = 'Chars_Raw', skiprows= 8, skipfooter =2, header = None, names=['Char', 'PortValue', 'IndValue'])

# Insert add'l columns for Seismic
chars.insert(0, 'AsOfDate', rpt_date)
chars.insert(1, 'StrategyCode', rpt_code)
chars.insert(2, 'Currency', rpt_curr)

# Preview
chars.head()

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,9/30/2019,MSAUM,AUD,# of Securities,29.000000,68.000000
1,9/30/2019,MSAUM,AUD,P/E (1Yr Trailing),21.897423,22.602774
2,9/30/2019,MSAUM,AUD,Dividend Yield,4.495234,4.527722
3,9/30/2019,MSAUM,AUD,EPS Growth (5Yr Trailing),10.189810,6.607627
4,9/30/2019,MSAUM,AUD,P/B,2.525590,2.505007


In [ ]:
# Rename chars derived from FDS to Seismic desired names

chars['Char'] = chars['Char'].replace('# of Securities', 'Number of Securities')
chars['Char'] = chars['Char'].replace('P/E (1Yr Trailing)', 'Price/Earnings Ratio (LTM)')
chars['Char'] = chars['Char'].replace('Dividend Yield', 'Dividend Yield (Current)')
chars['Char'] = chars['Char'].replace('EPS Growth (5Yr Trailing)', 'EPS Growth (5 Yr. Historical)')
chars['Char'] = chars['Char'].replace('P/B', 'Price/Book Ratio')
chars['Char'] = chars['Char'].replace('Market Cap - Weighted Median', 'Weighted Median Market Cap')
chars['Char'] = chars['Char'].replace('Market Cap - Weighted Average', 'Weighted Average Market Cap')
chars

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,9/30/2019,MSAUM,AUD,Number of Securities,29.000000,68.000000
1,9/30/2019,MSAUM,AUD,Price/Earnings Ratio (LTM),21.897423,22.602774
2,9/30/2019,MSAUM,AUD,Dividend Yield (Current),4.495234,4.527722
3,9/30/2019,MSAUM,AUD,EPS Growth (5 Yr. Historical),10.189810,6.607627
4,9/30/2019,MSAUM,AUD,Price/Book Ratio,2.525590,2.505007
5,9/30/2019,MSAUM,AUD,Weighted Average Market Cap,58918.464996,68383.029210
6,9/30/2019,MSAUM,AUD,Weighted Median Market Cap,27081.597656,45126.832031


In [ ]:
# Transform numerical output to appropriate type

item_1 = chars.loc[5,'PortValue']
item_2 = chars.loc[6,'PortValue']
item_3 = chars.loc[5, 'IndValue']
item_4 = chars.loc[6, 'IndValue']
item_5 = chars.loc[2,'PortValue']
item_6 = chars.loc[2, 'IndValue']
item_7 = chars.loc[3, 'PortValue']
item_8 = chars.loc[3, 'IndValue']
item_9 = chars.loc[1, 'PortValue']
item_10 = chars.loc[1, 'IndValue']
item_11 = chars.loc[4, 'PortValue']
item_12 = chars.loc[4, 'IndValue']

def MCap(item):
    item /= 1000
    item = item.round(1)
    item = str(item)
    item = currency_sign + item + ' B'
    return item

def PercString(item):
    item = "{:0.2f}%".format(item)
    return item

def RoundString(item):
    item = "{:0.2f}".format(item)
    return item

item_1 = MCap(item_1)
item_2 = MCap(item_2)
item_3 = MCap(item_3)
item_4 = MCap(item_4)
item_5 = PercString(item_5)
item_6 = PercString(item_6)
item_7 = PercString(item_7)
item_8 = PercString(item_8)
item_9 = RoundString(item_9)
item_10 = RoundString(item_10)
item_11 = RoundString(item_11)
item_12 = RoundString(item_12)


chars.loc[5,'PortValue'] = item_1
chars.loc[6,'PortValue'] = item_2 
chars.loc[5,'IndValue'] = item_3 
chars.loc[6, 'IndValue'] = item_4
chars.loc[2,'PortValue'] = item_5
chars.loc[2, 'IndValue'] = item_6
chars.loc[3, 'PortValue'] = item_7
chars.loc[3, 'IndValue'] = item_8
chars.loc[1, 'PortValue'] = item_9
chars.loc[1, 'IndValue'] = item_10
chars.loc[4, 'PortValue'] = item_11
chars.loc[4, 'IndValue'] = item_12

chars

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,9/30/2019,MSAUM,AUD,Number of Securities,29.0,68.0
1,9/30/2019,MSAUM,AUD,Price/Earnings Ratio (LTM),21.90,22.60
2,9/30/2019,MSAUM,AUD,Dividend Yield (Current),4.50%,4.53%
3,9/30/2019,MSAUM,AUD,EPS Growth (5 Yr. Historical),10.19%,6.61%
4,9/30/2019,MSAUM,AUD,Price/Book Ratio,2.53,2.51
5,9/30/2019,MSAUM,AUD,Weighted Average Market Cap,A$58.9 B,A$68.4 B
6,9/30/2019,MSAUM,AUD,Weighted Median Market Cap,A$27.1 B,A$45.1 B


### Step 4:  Wrangle and transform `Top Bottom 10` data from FactSet

In [ ]:
# Import Top 10 Portfolio Holdings by Weight tab and create df

tb = pd.read_excel('input_template.xlsx', sheet_name = 'TB_Raw', header = None, skiprows = 8, skipfooter = 1, names=['Holding', 'Weight'])

# Drop blank rows
tb = tb.dropna(axis=0)

# Delete Total Row
tb = tb[tb['Holding'] != 'Total']

# Sum up total of the top 10 holdings weights
ten_wgt_sum = tb['Weight'].sum()

# Insert add'l columns need for Seismic
tb.insert(0, 'AsOfDate', rpt_date)
tb.insert(1, 'StrategyCode', rpt_code)
tb.insert(4, 'WeightSum', ten_wgt_sum)

# Divide Weights by 100
tb[['Weight','WeightSum']] = tb[['Weight', 'WeightSum']].div(100)

tb

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Holding,Weight,WeightSum
0,9/30/2019,MSAUM,BHP Group Ltd,0.071344,0.564238
1,9/30/2019,MSAUM,Woolworths Group Ltd,0.066595,0.564238
2,9/30/2019,MSAUM,Commonwealth Bank of Australia,0.066324,0.564238
3,9/30/2019,MSAUM,Transurban Group Ltd.,0.063470,0.564238
4,9/30/2019,MSAUM,Rio Tinto Limited,0.054846,0.564238
5,9/30/2019,MSAUM,Newcrest Mining Limited,0.052735,0.564238
6,9/30/2019,MSAUM,Goodman Group,0.052517,0.564238
7,9/30/2019,MSAUM,Brambles Limited,0.048688,0.564238
8,9/30/2019,MSAUM,Telstra Corporation Limited,0.043924,0.564238
9,9/30/2019,MSAUM,Fortescue Metals Group Ltd,0.043795,0.564238


### Step 5:  Output to on single excel file

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('output_file.xlsx', engine = 'xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sector', index=False)
mcap.to_excel(writer, sheet_name='MCap', index=False)
chars.to_excel(writer, sheet_name='Chars', index = False)
tb.to_excel(writer, sheet_name='TB', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook = writer.book
worksheet_1 = writer.sheets['Sector']
worksheet_2 = writer.sheets['MCap']
worksheet_3 = writer.sheets['Chars']
worksheet_4 = writer.sheets['TB']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '0.00%'})
format2 = workbook.add_format()

worksheet_1.set_column('D:E', 18 , format1)
worksheet_1.set_column('A:C', 24, format2)
worksheet_2.set_column('F:G', 18 , format1)
worksheet_2.set_column('A:E', 18, format2)
worksheet_3.set_column('D:D', 30, format2)
worksheet_3.set_column('A:C',18, format2)
worksheet_3.set_column('E:F',18, format2)
worksheet_4.set_column('A:C', 18, format2)
worksheet_4.set_column('D:E', 18, format1)

writer.save()

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:\Characteristics\Flat File\Output\Flat_File_Chars_Weights.xlsx'